## ИДЗ1 Функциональный анализ. гр.9383 Ноздрин В.Я. 

### Теория  

В данной работе реализована функция **norm**, которая вычисляет норму Минковского по многограннику,  
заданому точками $A = \begin{bmatrix}
    a_{11} & a_{12} & 0 \\
    a_{21} & 0 & a_{23} \\
    0 & a_{32} & a_{33} \\
    a_{41} & 0 & 0 \\
    0 & a_{52} & 0 \\
    0 & 0 & a_{63}
\end{bmatrix}$

Для вычисления нормы точки $x$ необходимо:
1. Преобразовать точку так, чтобы она лежала в положительной области пространства, то есть нужно заменить все ее координаты на положительные.
2. Перебрать все конусы, задаваемые многогранником и для каждого конуса $V_1V_2V_3$ вычислить:  
    1. Вектора $N_1N_2N_3$ - биортогональный базис.
    2. Построим $n_i = N_i\frac{1}{(V_i,N_i)}$ чтобы $(n_i,V_i)=1$.
    3. Находим скалярные произведения векторов $n_1n_2n_3$ на вектор, соответствующий нашей точке результаты обозначим как $k_1k_2k_3$.

По построению, полученные произведения $k_1k_2k_3$ соответствуют коэффициентам разложения $\vec{x}=k_1\vec{V_1}+k_2\vec{V_2}+k_3\vec{V_3}$. Как только в одном из конусов все три координаты $k_1k_2k_3$ положительны, эти координаты принимаются как окончательное разложение, а число $k_1+k_2+k_3$ будет являтся нормой вектора $\vec{x}$

### Вариант 12

Для многогранника, заданного матрицей  
$V=\begin{bmatrix}10&7&0\\9&0&11\\0&8&12\\16&0&0\\0&11&0\\0&0&16\end{bmatrix}$ и точки $(35,46,19)$, коэффициенты $k_1k_2k_3$ становятся положительными на конусе $(10,7,0)(0,8,12)(0,11,0)$.
Биортогональный базис вычисляется как соответствующие векторные произведения например, $\vec{N_1}=\vec{V_2}\cdot\vec{V_3}$.
Таким образом получаем $$N_1=(-132,0,0),N_2=(0,0,110),N_3=(84,-120,80)$$
Нормируем эти вектора, получим $$\vec{n_1}=(0.1,0,0),\vec{n_2}=(0,0,0.084),\vec{n_3}=(-0.064,0.09,-0.06)$$ и далее, находя скалярные произведения векторов $\vec{n_1},\vec{n_2},\vec{n_3}$ на $\vec{x}$, получаем коэффициенты $k_1=3.5,k_2=1.58,k_3=0.8$.  
Итоговое разложение вектора $\vec{x}$ по базису $(10,7,0)(0,8,12)(0,11,0)$ будет выглядеть следующим образом: $$(35,46,19)=3.5\cdot(10,7,0)+1.58\cdot(0,8,12)+0.8\cdot(0,11,0)$$, а норма будет равна $||x||=5.886363636363636$

### Приложение. Код программы

In [18]:
import math,random as rm, time

#### Метод, генерирующий многогранник.

In [19]:
def generatePolyhedron(rmin=1, rmax=10, Rmin=5, Rmax=15):
    rm.seed(time.time())
    A = {
        "11": 0, "12": 0,             # A1 = a11 a12 0
        "21": 0, "23": 0,             # A2 = a21 0 a23
        "32": 0, "33": 0,             # A3 = 0 a32 a33
        "41": rm.randint(Rmin, Rmax), # A4 = a41 0 0
        "52": rm.randint(Rmin, Rmax), # A5 = 0 a52 0
        "63": rm.randint(Rmin, Rmax), # A6 = 0 0 a63
    }
    k21 = A["52"] / A["41"]
    A["11"] = int(rm.randint(rmin, A["41"]))
    A["12"] = int(rm.randint(math.ceil(k21*(A["41"]-A["11"])), A["52"]))
    k13 = A["41"] / A["63"]
    A["23"] = int(rm.randint(rmin, A["63"]))
    A["21"] = int(rm.randint(int(k13*(A["63"]-A["23"])), A["41"]))
    k32 = A["63"] / A["52"]
    A["32"] = int(rm.randint(rmin, A["52"]))
    A["33"] = int(rm.randint(int(k32*(A["52"]-A["32"])), A["63"]))
    ptsArray = [
        [A["11"],A["12"],0],
        [A["21"],0,A["23"]],
        [0,A["32"],A["33"]],
        [A["41"],0,0],
        [0,A["52"],0],
        [0,0,A["63"]]
    ]
    return ptsArray

#### Методы векторного произведения и скалярного произведения.

In [20]:
def CrossProduct(x, y):
    return [
        x[1] * y[2] - x[2] * y[1],
        x[2] * y[0] - x[0] * y[2],
        x[0] * y[1] - x[1] * y[0]
    ]


def DotProduct(x,y):
    return x[0]*y[0]+x[1]*y[1]+x[2]*y[2]

#### Метод, вычисляющий норму точки по многограннику.

In [31]:
def norm(point, poly=None):
    try:
        x, y, z = point
        x, y, z = map(abs,[x,y,z])
    except ValueError:
        raise ValueError("point must be from 3-dimentional space")
    if poly == None:
        return math.sqrt(x*x+y*y+z*z)
    cones = [
        [poly[0],poly[1],poly[2]],
        [poly[1],poly[0],poly[3]],
        [poly[0],poly[2],poly[4]],
        [poly[2],poly[1],poly[5]]
    ]
    for V in cones:
        N = [CrossProduct(V[2-1],V[3-1]),CrossProduct(V[1-1],V[3-1]),CrossProduct(V[1-1],V[2-1])]
        n = list(map(lambda a: list(map(lambda b: b/DotProduct(a[0], a[1]), a[0])), zip(N,V)))
        k = list(map(lambda a: DotProduct(a, point), n))
        should_be_x = list(map(lambda x: sum(map(lambda y: k[y]*V[y][x],range(len(k)))),range(len(k))))
        print(f"V:{V}\nN:{N}\nn:{n}\nk:{k}")
        print(f"{point}={k[0]}{V[0]}+{k[1]}{V[1]}+{k[2]}{V[2]}\t(={should_be_x})\n")
        if all(map(lambda x: x>=0, k)):
            return sum(k)
    return -1
    

#### Вычисления

In [29]:
# poly = generatePolyhedron()
poly = [[10, 7, 0], [9, 0, 11], [0, 8, 12], [16, 0, 0], [0, 11, 0], [0, 0, 16]]
print(poly)

[[10, 7, 0], [9, 0, 11], [0, 8, 12], [16, 0, 0], [0, 11, 0], [0, 0, 16]]


In [26]:
point = [rm.randint(0, 100),rm.randint(0, 100),rm.randint(0, 100)]
# point = [65, 97, 4]
print(point)

[35, 46, 19]


In [32]:
print(norm(point, poly))

V:[[10, 7, 0], [9, 0, 11], [0, 8, 12]]
N:[[-88, -108, 72], [84, -120, 80], [77, -110, -63]]
n:[[0.05378973105134474, 0.06601466992665037, -0.044009779951100246], [0.05134474327628362, -0.07334963325183375, 0.0488997555012225], [-0.04706601466992665, 0.06723716381418093, 0.03850855745721271]]
k:[4.083129584352078, -0.6479217603911982, 2.1772616136919316]
[35, 46, 19]=4.083129584352078[10, 7, 0]+-0.6479217603911982[9, 0, 11]+2.1772616136919316[0, 8, 12]	(=[35.0, 46.0, 19.0])

V:[[9, 0, 11], [10, 7, 0], [16, 0, 0]]
N:[[0, 0, -112], [0, 176, 0], [-77, 110, 63]]
n:[[-0.0, -0.0, 0.09090909090909091], [0.0, 0.14285714285714285, 0.0], [0.0625, -0.08928571428571429, -0.05113636363636364]]
k:[1.7272727272727273, 6.571428571428571, -2.891233766233767]
[35, 46, 19]=1.7272727272727273[9, 0, 11]+6.571428571428571[10, 7, 0]+-2.891233766233767[16, 0, 0]	(=[34.999999999999986, 46.0, 19.0])

V:[[10, 7, 0], [0, 8, 12], [0, 11, 0]]
N:[[-132, 0, 0], [0, 0, 110], [84, -120, 80]]
n:[[0.1, -0.0, -0.0], [0.0, 